# The Structural Relationship of Childcare Expenses and Workforce Attachment
## Jesús Pacheco & Dave Foote

In [3]:
#imports
import numpy as np
import pandas as pd
import scipy.stats as sts
import math
from matplotlib import pyplot as plt
import requests
import scipy.optimize as opt
from statsmodels.discrete.discrete_model import Probit
import statsmodels.api as sm
import structural_childcare as sc
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [9]:
#read data
df = pd.read_csv('../rdf_subset.csv')
df['intercept'] = 1
df.intercept.head()

0    1
1    1
2    1
3    1
4    1
Name: intercept, dtype: int64

In [108]:
#our functions:
def mle_beta_vec(df, xcols, init_guess, target_col, f):
    '''
    df = dataframe
    xcolumn names in list form
    init_guess in tuple form
    criterion function f
    doin the damn thing (write better comment later)
    '''
    x_mat = extract_x_matrix(df, xcols)
    y_vec = df[target_col]
    
    results_uncstr = opt.minimize(f, init_guess, method = 'L-BFGS-B', args=(x_mat, y_vec))

    return results_uncstr

def criterion(beta_guess, *args):
    '''
    beta_guess comes in as array
    '''
    x_mat, y_vec = args
    
    
    
    return logit_neglog_likelihood(beta_guess, y_vec, x_mat, prob_1)

def logit_neglog_likelihood(beta_vec, *args):
    '''
    calculate the log likelihood that the probability is correct
    '''
    y_vec, xm, probability_now = args
    p = list(probability_now(xm, beta_vec))
    y_vec = list(y_vec)
    rv = []
    for i, x in enumerate(p):
        to_add = ((y_vec[i] * math.log(x))) + ((1 - y_vec[i]) * math.log(1 - x))
        if to_add is not np.nan:
            rv.append(to_add)
    rv = pd.Series(rv)

    return -(rv.sum())

def prob_1(x_matrix, beta_vec):
    '''
    calculate probability a set of observations is a member of d1, using logit
    classification
    '''
    linear_kernel = x_matrix.dot(beta_vec)
    rv = np.exp(linear_kernel) / (1 + np.exp(linear_kernel))
    rv[rv == 1] = .999999

    return rv

def categorical_split(df):
    cond1 = (df['monthly_childcare_expenditure'] == 0)
    cond2 = (df['monthly_childcare_expenditure'] > 0)
    cond3 = (df['monthly_wage'] > 0)
    cond4 = (df['monthly_wage'] == 0)

    return df[cond1 & cond3], df[cond2 & cond3], df[cond4]

def extract_x_matrix(df, xcols):
    '''
    inputs: df with data and column names of your x variables
    output: n x k matrix of x data where n is #observations and k is
    #of columns
    '''
    
    return df[xcols].to_numpy()

def categorize_ds(df):
    '''
    input: df with H and F calculated columns
    output: same df with d1-4 columns denoting membership in groups d1, d2,
    d3, and d4 defined as such:
    d1 - H = 0, F = 0
    d2 - H = 0, F = 1
    d3 - H = 1, F = 0
    d4 - H = 1, F = 1
    '''

    df['d1'] = [1 if (r['H'] == 0) & (r['F'] == 0) else 0 for i, r in df.iterrows()]
    df['d2'] = [1 if (r['H'] == 0) & (r['F'] == 1) else 0 for i, r in df.iterrows()]
    df['d3'] = [1 if (r['H'] == 1) & (r['F'] == 0) else 0 for i, r in df.iterrows()]
    df['d4'] = [1 if (r['H'] == 1) & (r['F'] == 1) else 0 for i, r in df.iterrows()]

In [109]:
categorize_ds(df)

In [110]:
results1 = mle_beta_vec(df, ['intercept', 'k_under2'], (0, .1), 'd1', criterion)

In [111]:
results1

      fun: 3940.769926996391
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.54747351e-05,  4.54747351e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 21
      nit: 5
   status: 0
  success: True
        x: array([-0.5548937 ,  0.49808676])

In [112]:
results2 = mle_beta_vec(df, ['intercept', 'k_under2'], (0, .1), 'd2', criterion)

In [113]:
results2

      fun: 1696.2057330776115
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.82121026e-05,  2.27373675e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 33
      nit: 10
   status: 0
  success: True
        x: array([-2.39043974, -0.01763054])

In [114]:
results3 = mle_beta_vec(df, ['intercept', 'k_under2'], (0, .1), 'd3', criterion)

In [115]:
results3

      fun: 3467.361392719559
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00113687, -0.00022737])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 7
   status: 0
  success: True
        x: array([-0.78912389, -0.49949474])

In [116]:
results4 = mle_beta_vec(df, ['intercept', 'k_under2'], (0, .1), 'd4', criterion)

In [117]:
results4

      fun: 3185.0444942783183
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00022737, 0.00022737])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 27
      nit: 8
   status: 0
  success: True
        x: array([-1.15331466, -0.16083496])